In [2]:
# colab에 KoNLPy 설치

!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy 
!JAVA_HOME="C:\Program Files\Java\jdk-14.0.2"


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Ign:13 https://developer.download.nvidia.com/comp

In [3]:
# 데이터 불러오기

import pandas as pd

data = pd.read_csv('preprocessed_data.csv', encoding='utf-8', index_col=0)

In [4]:
# 데이터 살펴보기

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77149 entries, 0 to 77148
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  70549 non-null  object
dtypes: object(1)
memory usage: 1.2+ MB


In [5]:
from konlpy.tag import Okt

def extract_noun(texts):
  okt = Okt()
  extract_corpus = []
  
  for line in texts:
    temp = okt.pos(str(line), norm=True, stem=True)
    nouns = []
    for i in range(len(temp)):
      if temp[i][1] == "Noun":
        nouns.append(temp[i][0])
    extract_corpus.append(nouns)

  return extract_corpus



In [6]:
# 네이버 영수증 리뷰에서 Noun만 뽑아서 extract_corpus와 비교

extract_corpus = extract_noun(data['Review'])

# 졸업논문 미아 박선익 찾습니다 

In [7]:
# 맞게 되었는지 이전의 데이터와 비교

print(len(data), len(extract_corpus))

77149 77149


In [20]:
text = extract_corpus.copy()

In [24]:
text[-20:]

[['바닐라', '젤라또', '양', '처음', '살짝', '쫄깃해', '칼', '안', '드립', '아이스커피'],
 ['인생', '용', '짱', '맛'],
 ['콜드', '브루', '드립', '커피', '향', '아주', '구', '웃'],
 [],
 ['젤라또', '젤라또', '인심', '아주', '다음', '젤라또', '양'],
 ['용'],
 [],
 [],
 [],
 [],
 ['맛', '요', '굿'],
 ['곳', '카페', '서비스', '커피', '맛'],
 ['결혼식', '커피', '디저트', '게'],
 ['커피', '곳', '애'],
 ['원두', '편이'],
 ['더치', '드립', '커피', '둘', '맛', '가격', '아주', '추천'],
 ['커피', '맛'],
 [],
 ['분위기', '맛'],
 ['원두', '종류', '제제', '나인']]

In [9]:
detokenized_doc = []

for i in range(len(text)):
  t = ' '.join(text[i])
  detokenized_doc.append(t)


In [10]:
detokenized_doc[:10]

['',
 '',
 '꼬끄 정말 쫀득 마카롱 크기 맛 요 선물 때 아주 것',
 '굿',
 '',
 '오늘 메뉴 마카롱 맛',
 '',
 '제이 엠티',
 '가성 비',
 '아들']

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=3)

vect = vectorizer.fit_transform(detokenized_doc)


In [12]:
vect.toarray().shape

(77149, 5063)

In [13]:
for i in range(1, 4):
  vectorizer = CountVectorizer(min_df=i)
  vect = vectorizer.fit_transform(detokenized_doc)
  print("min_df는 {}, shape는 {}".format(i, vect.toarray().shape))

min_df는 1, shape는 (77149, 12751)
min_df는 2, shape는 (77149, 6925)
min_df는 3, shape는 (77149, 5063)


In [14]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation()

In [15]:
lda.fit_transform(vect.toarray())

array([[0.1       , 0.1       , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       [0.1       , 0.1       , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       [0.01250002, 0.01250092, 0.43213967, ..., 0.1992537 , 0.11938064,
        0.0125    ],
       ...,
       [0.1       , 0.1       , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       [0.05000054, 0.05      , 0.05      , ..., 0.05      , 0.54999351,
        0.05000573],
       [0.02500005, 0.02500001, 0.02500012, ..., 0.02503323, 0.02500013,
        0.0250026 ]])

In [16]:
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda.components_,terms)



Topic 1: [('카페', 5015.94), ('케이크', 1517.2), ('아이스크림', 673.94), ('힐링', 498.83), ('와플', 488.1), ('풍경', 460.12), ('굿굿', 411.1), ('느낌', 409.06), ('공간', 237.56), ('인테리어', 227.22)]
Topic 2: [('맛집', 3279.58), ('가격', 1936.99), ('스콘', 526.07), ('완전', 467.66), ('식빵', 460.72), ('조금', 455.44), ('초코', 450.64), ('버터', 450.06), ('아인', 374.1), ('페너', 286.1)]
Topic 3: [('빵집', 967.03), ('빙수', 812.1), ('추천', 754.91), ('사장', 619.14), ('여기', 453.48), ('동네', 403.24), ('이드', 368.64), ('제일', 276.58), ('생각', 267.19), ('망고', 249.85)]
Topic 4: [('라떼', 1754.54), ('바다', 981.57), ('디저트', 896.38), ('카페', 841.98), ('마카롱', 448.1), ('바닐라', 375.48), ('딸기', 367.39), ('케잌', 296.21), ('친절', 236.27), ('크림', 233.84)]
Topic 5: [('그냥', 1040.43), ('아메리카노', 971.94), ('매장', 727.33), ('카페', 427.76), ('아이스', 405.97), ('하나', 395.35), ('메뉴', 313.6), ('주차', 309.17), ('포장', 282.74), ('평일', 282.66)]
Topic 6: [('음료', 2818.95), ('진짜', 2058.74), ('케익', 1019.55), ('가격', 720.88), ('아이', 428.79), ('가게', 413.36), ('디저트', 328.05), ('다른', 310.22